In [27]:
import pandas as pd
import pyodbc

# 
file_items = r"E:\Reyhane\Data for Project for the future\annex\New folder\annex1.xlsx"
file_sales = r"E:\Reyhane\Data for Project for the future\annex\New folder\annex2.xlsx"
file_wholesale = r"E:\Reyhane\Data for Project for the future\annex\New folder\annex3.xlsx"
file_lossrate = r"E:\Reyhane\Data for Project for the future\annex\New folder\annex4.xlsx"

# 
df_items = pd.read_excel(file_items)
df_sales = pd.read_excel(file_sales)
df_wholesale = pd.read_excel(file_wholesale)
df_lossrate = pd.read_excel(file_lossrate)
print(df_sales.columns)

# 
def clean_str(value):
    return value.strip() if isinstance(value, str) else value

def clean_numeric(value):
    return pd.to_numeric(value, errors='coerce')

# 
for df in [df_items, df_sales, df_wholesale, df_lossrate]:
    df.columns = [col.strip() for col in df.columns] 
    df.applymap(lambda x: clean_str(x) if isinstance(x, str) else x)

df_sales['Quantity Sold kilo'] = clean_numeric(df_sales['Quantity Sold kilo'])
df_sales['Unit Selling Price'] = clean_numeric(df_sales['Unit Selling Price'])
df_wholesale['Wholesale Price'] = clean_numeric(df_wholesale['Wholesale Price'])
df_lossrate['Loss Rate'] = clean_numeric(df_lossrate['Loss Rate'])

# 
df_sales.dropna(subset=['Quantity Sold kilo', 'Unit Selling Price'], inplace=True)
df_wholesale.dropna(subset=['Wholesale Price'], inplace=True)
df_lossrate.dropna(subset=['Loss Rate'], inplace=True)
df_sales.rename(columns={'Quantity Sold kilo': 'Quantity_Sold_kilo'}, inplace=True)
# 🔗 SQL Server
conn = pyodbc.connect(
    "Driver={ODBC Driver 17 for SQL Server};"
    "Server=DESKTOP-OK9UTDK;"
    "Database=annex;"
    "Trusted_Connection=yes;"
)
cursor = conn.cursor()
# 
cursor.execute("""
IF NOT EXISTS (SELECT * FROM sysobjects WHERE name='Items' AND xtype='U')
CREATE TABLE Items (
    Item_Code VARCHAR(50) PRIMARY KEY,
    Item_Name NVARCHAR(255),
    Category_Code VARCHAR(50),
    Category_Name NVARCHAR(255)
)
""")

cursor.execute("""
IF NOT EXISTS (SELECT * FROM sysobjects WHERE name='Sales' AND xtype='U')
CREATE TABLE Sales (
    Sale_ID INT IDENTITY(1,1) PRIMARY KEY,
    Sale_Date DATE,
    Sale_Time TIME,
    Item_Code VARCHAR(50),
    Quantity_Sold_kilo FLOAT,
    Unit_Selling_Price FLOAT,
    Sale_or_Return VARCHAR(10),
    Discount VARCHAR(3),
    FOREIGN KEY (Item_Code) REFERENCES Items(Item_Code)
)
""")

cursor.execute("""
IF NOT EXISTS (SELECT * FROM sysobjects WHERE name='Wholesale' AND xtype='U')
CREATE TABLE Wholesale (
    Wholesale_Date DATE,
    Item_Code VARCHAR(50),
    Wholesale_Price FLOAT,
    PRIMARY KEY (Wholesale_Date, Item_Code),
    FOREIGN KEY (Item_Code) REFERENCES Items(Item_Code)
)
""")

cursor.execute("""
IF NOT EXISTS (SELECT * FROM sysobjects WHERE name='LossRate' AND xtype='U')
CREATE TABLE LossRate (
    Item_Code VARCHAR(50) PRIMARY KEY,
    Item_Name NVARCHAR(255),
    Loss_Rate FLOAT,
    FOREIGN KEY (Item_Code) REFERENCES Items(Item_Code)
)
""")

conn.commit()


# 
cursor.execute("DELETE FROM Sales")
cursor.execute("DELETE FROM Wholesale")
cursor.execute("DELETE FROM LossRate")
cursor.execute("DELETE FROM Items")
conn.commit()

# 

# ✅  Items
for _, row in df_items.iterrows():
    cursor.execute("""
        INSERT INTO Items (Item_Code, Item_Name, Category_Code, Category_Name) 
        VALUES (?, ?, ?, ?)
    """, row['Item Code'], row['Item Name'], row['Category Code'], row['Category Name'])

# ✅  Sales
for _, row in df_sales.iterrows():
    cursor.execute("""
    INSERT INTO Sales (Sale_Date, Sale_Time, Item_Code, Quantity_Sold_kilo, Unit_Selling_Price, Sale_or_Return, Discount) 
    VALUES (?, ?, ?, ?, ?, ?, ?)
""", row['Sale Date'], row['Sale Time'], row['Item Code'], row['Quantity_Sold_kilo'], 
   row['Unit Selling Price'], row['Sale or Return'], row['Discount'])


# ✅  Wholesale
for _, row in df_wholesale.iterrows():
    cursor.execute("""
        INSERT INTO Wholesale (Wholesale_Date, Item_Code, Wholesale_Price) 
        VALUES (?, ?, ?)
    """, row['Wholesale Date'], row['Item Code'], row['Wholesale Price'])

# ✅  LossRate
for _, row in df_lossrate.iterrows():
    cursor.execute("""
        INSERT INTO LossRate (Item_Code, Item_Name, Loss_Rate) 
        VALUES (?, ?, ?)
    """, row['Item Code'], row['Item Name'], row['Loss Rate'])

conn.commit()
conn.close()

print("✅All data was successfully saved to SQL!")


Index(['Sale Date', 'Sale Time', 'Item Code', 'Quantity Sold kilo',
       'Unit Selling Price', 'Sale or Return', 'Discount'],
      dtype='object')


C:\Users\Reyhane\AppData\Local\Temp\ipykernel_20196\411048884.py:27: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df.applymap(lambda x: clean_str(x) if isinstance(x, str) else x)
C:\Users\Reyhane\AppData\Local\Temp\ipykernel_20196\411048884.py:27: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df.applymap(lambda x: clean_str(x) if isinstance(x, str) else x)
C:\Users\Reyhane\AppData\Local\Temp\ipykernel_20196\411048884.py:27: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df.applymap(lambda x: clean_str(x) if isinstance(x, str) else x)


✅ همه داده‌ها با موفقیت در SQL ذخیره شدند!
